<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Experiments and Hypothesis Testing

_Authors: Alexander Egorenkov (DC)_

---

<a id="learning-objectives"></a>
### Learning Objectives
- Explain the difference between causation and correlation.
- Determine causality and sampling bias using directed acyclic graphs.
- Identify what missing data is and how to handle it.
- Test a hypothesis using a sample case study.

### Lesson Guide
- [Data Source](#data-source)
	- [What Are the Features/Covariates/Predictors?](#what-are-the-featurescovariatespredictors)
	- [What Is the Outcome/Response?](#what-is-the-outcomeresponse)
	- [What Do You Think Each Row in the Data Set Represents?](#what-do-you-think-each-row-in-the-dataset-represents)
- [Math Review](#math-review)
	- [Covariance](#covariance)
	- [Correlation](#correlation)
	- [The Variance-Covariance Matrix](#the-variance-covariance-matrix)
- [Causation and Correlation](#causation-and-correlation)
	- [Structure of Causal Claims](#structure-of-causal-claims)
	- [Why Do We Care?](#why-do-we-care)
	- [How Do We Determine if Something is Causal?](#how-do-we-determine-if-something-is-causal)
- [The Pearlean Causal DAG Model](#pearlean-causal-dag-model)
	- [What Is a DAG?](#what-is-a-dag)
	- [X Causes Y](#its-possible-that-x-causes-y)
	- [Y Causes X](#y-causes-x)
	- [The Correlation Between X and Y Is Not Statistically Significant](#the-correlation-between-x-and-y-is-not-statistically-significant)
	- [X or Y May Cause One or the Other Indirectly Through Another Variable](#x-or-y-may-cause-one-or-the-other-indirectly-through-another-variable)
	- [There is a Third Common Factor That Causes Both X and Y](#there-is-a-third-common-factor-that-causes-both-x-and-y)
	- [X and Y Cause a Third Factor, But Our Data Collect the Third Factor Unevenly](#both-x-and-y-cause-a-third-variable-and-the-dataset-does-not-represent-that-third-variable-evenly)
	- [Controlled Experiments](#controlled-experiments)
	- [When Is it OK to Rely on Association?](#when-is-it-ok-to-rely-on-association)
	- [How Does Association Relate to Causation?](#how-does-association-relate-to-causation)
- [Sampling Bias](#sampling-bias)
	- [Forms of Sampling Bias](#forms-of-sampling-bias)
	- [Problems From Sampling Bias](#problems-from-sampling-bias)
	- [Recovering From Sampling Bias](#recovering-from-sampling-bias)
    - [Stratified Random Sampling](#stratified-random-sampling)
- [Missing Data](#missing-data)
	- [Types of Missing Data](#types-of-missing-data)
	- [De Minimis](#de-minimis)
	- [Class Imbalance](#class-imbalance)
    - [Relation to Machine Learning](#relation-to-machine-learning)
- [Introduction to Hypothesis Testing](#introduction-to-hypothesis-testing)
	- [Validate Your Findings](#validate-your-findings)
	- [Confidence Intervals](#confidence-intervals)
	- [Error Types](#error-types)
- [Scenario](#scenario)
	- [Exercises](#exercises)
	- [Statistical Tests](#statistical-tests)
	- [Interpret Your Results](#interpret-your-results)

## Scientific Method

**1. Question**<br>
**2. Hypothesis**<br>
**3. Experiment**<br>
**4. Observation**<br>
**5. Analysis**<br>
**6. Conclusion**<br>
<br>

### Rules of Experiments
1. Must show that a hypothesis is **Supported** or **Not Supported**
2. Results must be **Measurable** and **Objective**
3. Must be **Repeatable**.

<a id="data-source"></a>
## Data Source

---

Today, we’ll use advertising data from an example in the book [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/).
- This is a well-known, standard introduction to machine learning.
- The book has a more advanced version — [Elements of Statistical Learning](http://web.stanford.edu/~hastie/ElemStatLearn/) — if you are comfortable with linear algebra and statistics at the graduate level.

#### Code-Along: Bring in Today's Data

In [ ]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# This allows plots to appear directly in the notebook.
%matplotlib inline
plt.style.use('fivethirtyeight') 

In [ ]:
# Read data into a DataFrame.

# We use index_col to tell Pandas that the first column in the data has row labels.
advertising = pd.read_csv('./data/advertising.csv')
advertising.head() 

In [ ]:
# Examine the data with .head(). 

#### Questions About the Advertising Data

Let's pretend you work for the company that manufactures and markets this new device. The company might ask you the following: "On the basis of this data, how should we spend our advertising money in the future?"

<a id="what-are-the-featurescovariatespredictors"></a>
### What are the Features/Covariates/Predictors?

In [ ]:
# Answer:

<a id="what-is-the-outcomeresponse"></a>
### What Is the Outcome/Response?

In [ ]:
# Answer:

<a id="what-do-you-think-each-row-in-the-dataset-represents"></a>
### What Do You Think Each Row in the Data Set Represents?

In [ ]:
# Answer:

<a id="math-review"></a>
## Math Review
---

<a id="covariance"></a>
### Covariance

Covariance is a measure of the joint variability between two random variables.

You can think of this as a measure of linear association. If you have a variance of Y and a variance of X, the covariance is the amount of variance they share.

$$cov(X, Y) = \frac {\sum{(x_i - \bar{X})(y_i - \bar{Y})}} {n}$$

#### The math can be a bit intimidating, but I show it only to ask these two questions:

* When will covariance be positive?
* How will outliers affect covariance?

> We can gain insight into covariance by looking closely at the formula above. First, observe that the formula effectively pairs the first $x$ data point with the first $y$ data point: $(x_1, y_1)$. All computations are done solely on these pairs of points.

> Second, let's ask ourselves, **when would covariance be positive**? From the numerator, covariance would be positive if, for all pairs of data points, $(x_i - \bar{X})$ and $(y_i - \bar{Y})$ are 1) both positive or 2) both negative. This occurs when: 1) Both data points are greather than their respective means. Or when: 2) Both data points are less than their respective means! So, if the $x$ data points vary from their mean in the same way the $y$ data points vary from their mean, covariance will be positive.

> Third, let's consider: **Might outliers affect covariance?** Yes! Given the structure of the formula (a sum of terms), a large outlier pair far from the means could strongly pull the covariance in one direction.


**A Useful Special Case (Used Below)**

$$cov(X, X) = \frac {\sum{(x_i - \bar{X})^2}} {n} = var(X) = \sigma_X^2$$


### Principles of Covariance
1. **Bilinearity.** Covariance is directly proportional to the scale of X and Y.

2. **Correlation.** Covariance increases as the points approximate an upward sloping line and decreases as the points approximate a downward sloping line.

3. **Linear Associations.** Because non-linear associations can create mixtures of positives and negatives, they lead to unpredictable (and not very useful) covariances.

4. **Outlier Sensitivity.** Given the structure of the formula (a sum of terms), a large outlier pair far from the means could strongly pull the covariance in one direction.

<a id="correlation"></a>
### Correlation

While covariance is a useful measure, it can be difficult to compare covariances, as they are not standardized. 

Instead we can use the correlation, which measures the same effect but reports it as a range from -1 to 1. 1 represents perfect covariance and correlation, 0 represents no correlation, and -1 one represents perfect inverse correlation.

$$corr(X,Y) = \frac {cov(X,Y)} {\sigma_X\sigma_Y} = \frac {\mathbb{E}[(\mathbf{X}-\mathbb{E}[\mathbf{X}])(\mathbf{Y}-\mathbb{E}[\mathbf{Y}])]} {\sigma_X\sigma_Y}$$

Note that the variance is always positive, making the denominator positive. So, the sign of the covariance between $X$ and $Y$ is the same as the sign of their correlation! 

The following visual examples better illustrate how correlation refers to how $X$ and $Y$ change together. Notice that a correlation number by itself is not always indicative of the relationship between the variables — always try to supplement 2-D correlation with a visual!

![](./assets/images/correlation_examples.png)


Another familiar way to look at correlation, with R-squared (Note: The square root of R-squared is the correlation coefficient):

<img src="./assets/images/Correlation.jpg">

<a id="the-variance-covariance-matrix"></a>
### The Variance-Covariance Matrix

For our purposes in modeling and machine learning, the fastest way to get a preview of the underlying relationships in our data is to use the variance-covariance matrix.

The variance-covariance matrix shows the covariance between every variable in our data set.

If you have many $x$-variables, it's common to organize covariances into a **var-covar matrix** (sometimes just called a **covariance matrix**).

Given $n$ features from $X_1$ to $X_n$, the variance-covariance matrix looks like this (recall that $cov(X, X) = var(X)$):

$$
\mathbf{\Sigma} = 
\left[ \begin{array}{c}
\text{Var}(X_1) & \text{Cov}(X_1,X_2) & \cdots & \text{Cov}(X_1,X_n)  \\
\text{Cov}(X_2,X_1) & \text{Var}(X_2) & \cdots & \text{Cov}(X_2,X_n)  \\
\vdots & \vdots & \ddots & \vdots \\
\text{Cov}(X_n,X_1) & \text{Cov}(X_n,X_2) & \cdots & \text{Var}(X_n)
\end{array} \right]
$$


From a quick glance at this matrix, we can glean insight about which variables might be strongly correlated. This may also indicate redundant features and/or affect some models.

If data are centered around the mean, every column has its mean subtracted from itself. So, the mean for every column is now 0. You can then compute the variance-covariance matrix as:

$$\frac {X^TX} {n}$$

Those of you who have been exposed to linear regression may recognize this term.

But... still not useful right? We can't read covariances easily. So it's also common to have a **correlation matrix**:

$$
\mathbf{R} = 
\begin{bmatrix}
1 & \text{Corr}(X_1, X_2) & \cdots & \text{Corr}(X_1, X_n) \\
\text{Corr}(X_2, X_1) & 1 & \cdots & \text{Corr}(X_2, X_n) \\
\vdots & \vdots & \ddots & \vdots \\
\text{Corr}(X_n, X_1) & \text{Corr}(X_n, X_2) & \cdots & 1 \\
\end{bmatrix}
$$

#### Calculate the variance-covariance matrix. Make sure to first de-mean the data:

##### Or.... use the DataFrame's built-in .cov() method

In [ ]:
# Answer:
X= advertising - advertising.mean()
np.dot(X.T, X) / ((X.shape[0])-1)

In [ ]:
advertising.cov()

#### Calculate the correlation matrix using the DataFrame's built-in `.corr()` method:

In [ ]:
# Answer:
advertising.corr()

When we have a large amount of data, the correlation matrix may be too difficult to read. It can help to make a plot.

#### Use Seaborn's `.heatmap()` function to make a plot of the correlation matrix.

- Remember that we imported Seaborn as `sns`.
- To make a correlation matrix from a DataFrame, try `my_df.corr()`.

In [ ]:
# Answer:
sns.heatmap(advertising.corr(), cmap='coolwarm')

Or, use the Seaborn `.clustermap` method

In [ ]:
sns.clustermap(advertising.corr(), cmap='coolwarm')

Of course, looking at linear association doesn't show us the whole picture. We can get a more detailed look with a scatterplot matrix.

#### Use Seaborn's `.pairplot()` function to make joint scatterplots of the data.

- See if you can guess or [figure out](http://seaborn.pydata.org/generated/seaborn.pairplot.html) how `pairplot()` might work.
- `pairplot()` plots each column against each column of a DataFrame. So, at the minimum you must have to pass in the DataFrame you want to analyze!

In [ ]:
# Answer:
sns.pairplot(advertising)

<a id="causation-and-correlation"></a>
## Causation and Correlation
---

**Objective**: Explain the difference between causation and correlation.

- Think of various examples of studies you’ve seen in the media related to food:
    - "Study links coffee consumption to decreased risk of colorectal cancer"
    - "Coffee does not decrease risk of colorectal cancer"
    - "Diapers, Beer and Data Science in Retail"
    
There's a whole book series based on these [Spurious Correlations](http://www.tylervigen.com/spurious-correlations).

**So, why are these spurious correlations so common?**

- Sensational Headlines
- Professor's need to publish, publish, publish!
- There's a neglect of robust data analysis.
- Causal claims and associations are difficult to convey in an unambiguous way.

The coffee claims above are **correlated** but may or may not be **causal**.

<a id="structure-of-causal-claims"></a>
### Structure of Causal Claims

- If X happens, Y must happen.
- If Y happens, X must have happened.
  - (You need X and something else for Y to happen.)
- If X happens, Y will probably happen.
- If Y happens, X probably happened.

> **Note:** Properties from definitions are not causal. If some a shape is a triangle, it's implied that it has three sides. However, it being a triangle does not _cause_ it to have three sides.

<a id="why-do-we-care"></a>
### Why Do We Care?

- Understanding this difference is critical for executing the data science workflow, especially when identifying and acquiring data.
- We need to fully articulate our question and use the right data to answer it while also considering any **confounders**.

> **Confounders** are unobserved variables that could affect the outcome. If we neglect to include confounding variables in an analysis, we could easily produce an inaccurate model. For example, we might falsely assume that eating more ice cream cones causes us to wear fewer layers of clothing. In actuality, eating ice cream is correlated with a confounding variable — temperature! To perform an accurate analysis, we can only conclude that ice cream consumption is _correlated with_ clothing layers.

- We don’t want to overstate what our model measures.
- Be careful not to say “caused” when you really mean “measured” or “associated.”

<a id="how-do-we-determine-if-something-is-causal"></a>
### How Do We Determine if Something is Causal?

Considering causal criteria is one approach to assessing causal relationships.

However, it’s hard to define universal causal criteria.

One attempt that's commonly used in the medical field is based on work by Bradford Hill.


He developed a list of “tests” that an analysis must pass in order to indicate a causal relationship:


- Strength of association
- Consistency
- Specificity
- Temporality
- Biological gradient
- Plausibility
- Coherence
- Experiment
- Analogy


**Strength (effect size)**: A small association does not mean that there is not a causal effect, although the larger the association, the more likely the effect is to be causal.

**Consistency (reproducibility)**: Consistent findings observed by different persons in different places with different samples strengthens the likelihood of an effect.

**Specificity**: Causation is likely if there is a very specific population at a specific site and a disease with no other likely explanation. The more specific an association between a factor and an effect, the greater the probability of a causal relationship.

**Temporality**: The effect has to occur after the cause (and, if there is an expected delay between the cause and expected effect, then the effect must occur after that delay).

**Biological gradient**: Greater exposure should generally lead to greater incidence of the effect. However, in some cases, the mere presence of the factor can trigger the effect. In other cases, an inverse proportion is observed: greater exposure leads to lower incidence.

**Plausibility**: A plausible mechanism between cause and effect is helpful (but Hill noted that knowledge of the mechanism is limited by current knowledge).

**Coherence**: Coherence between epidemiological and laboratory findings increases the likelihood of an effect. However, Hill noted that "... lack of such [laboratory] evidence cannot nullify the epidemiological effect on associations."

**Experiment**: "Occasionally it is possible to appeal to experimental evidence."

**Analogy**: The effect of similar factors may be considered.

<a id="pearlean-causal-dag-model"></a>
## The Pearlean Causal Directed Acyclic Graph (DAG) Model

---
### Some Quick Background Notes:

- This model is a visual tool to help us reason about causality and association.
- It was proposed by Judea Pearl, although there are many similar models.
- We will only scratch the surface, so look into other resources if you're interested in learning more.
    - We'll cover the basic idea and most notable cases.
    - We won't talk about the formal mathematics or underlying probability, or how to use d-seperation to infer causality.

<a id="what-is-a-dag"></a>
### What Is a DAG?
- DAG stands for directed acyclic graph; it's a collection of nodes connected by lines. 
- Each line has an arrow to point in a direction.
- If you follow the arrows, you reach a final node. There are no loops.

A single circle or node in a causal DAG represents an event; something that happens at one point in time.

![](./assets/images/dag1.png)

Let's pretend random variables X and Y — or two different types of events — are correlated with each other.

**What are the possible causal structures that would produce this correlation?**

- X causes Y.
- Y causes X.
- There is no actual causation.
- X or Y indirectly causes the other.
- There is a third factor that causes both.
- X and Y cause a third factor, but our data collect the third factor unevenly.

<a id="its-possible-that-x-causes-y"></a>
### X causes Y.
![](./assets/images/x-cause-y.png)

<a id="y-causes-x"></a>
### Y causes X.
![](./assets/images/y-cause-x.png)

<a id="the-correlation-between-x-and-y-is-not-statistically-significant"></a>
### The correlation between X and Y is not statistically significant.
![](./assets/images/xy.png)

<a id="x-or-y-may-cause-one-or-the-other-indirectly-through-another-variable"></a>
### X or Y may cause one or the other indirectly through another variable.
![](./assets/images/x-c-z-y.png)

<a id="there-is-a-third-common-factor-that-causes-both-x-and-y"></a>
### There is a third common factor that causes both X and Y.
![](./assets/images/z-cause-xy.png)

<a id="both-x-and-y-cause-a-third-variable-and-the-dataset-does-not-represent-that-third-variable-evenly"></a>
### X and Y cause a third factor, but our data collect the third factor unevenly.

![](./assets/images/xy-causez.png)

Generally, recovering the causality structure from a correlation matrix is difficult or at times impossible. However, thinking through causal effects can give you a much better intuition regarding your variables and your data.

### What Is a "Confounder"?

Let’s say we performed an analysis to understand what causes lung cancer. 

We find that people who carry cigarette lighters are 2.4 times more likely to contract lung cancer than people who don’t carry lighters.

Does this mean that the lighters are causing cancer?

As we've noted before, if lighters and cancer are both caused by smoking, there will be a correlation between lighters and cancer. This isn't the only possible diagram, but it makes the most sense.
![](./assets/images/smoke-lighter-cancer.png)

If we believe the structure above, conditioning on smoking by only looking at non-smokers removes the correlation between lighters and cancer.

<a id="controlled-experiments"></a>
### Controlled Experiments

- The most foolproof way to measure an effect is to control all of the confounders and directly intervene and control our variable of interest. 
- This way we know that any correlation we find is not because of the confounders but instead because of the variable we control. 
- This also means that all the effects we see are due to the variable we control.
- However, experiments are not always possible and take longer than using observational data.

<a id="when-is-it-ok-to-rely-on-association"></a>
### When Is it OK to Rely on Association?

- **When any intervention that arises from your model affects only the outcome variable.**
    - In other words, you only need to predict Y.
    - This works because we only need to observe explanatory variables and implicitly know the confounders' effect.
    - Decision-making and intervention based on your model are hidden dangers that can shift confounders.
    - You can always retrain your model to work with a new set of confounders if they shift.

- **When correlation is causal.**
    - If you are sure there are no confounding factors or selection bias, then that association might be a causation (risky).
    - It's OK to exclude confounders that have very unlikely or small effects.
    - This is a saving grace. To create a good model, you only need variables that correlate with your outcome.
        - Those variables merely need to meaningfully correlate with your outcome.

<a id="how-does-association-relate-to-causation"></a>
### How Does Association Relate to Causation?

- Most commonly, we find an association between two variables.
    - There is an observed correlation between the variables.
    - There is an observed correlation in a subset of data.
    - We find that the descriptive statistics significantly differ in two subsets of data.

- We may not still fully understand the causal direction (e.g., does smoking cause cancer or does cancer cause smoking?).
    - A causes B, B causes A, or a third factor causes both.
        - A and B never cause each other!

- We also might not understand other factors influencing the association.

Confounding variables often hide the true association between causes and outcomes.

A directed acyclic graph (DAG) can help determine which variables are most important for your model. It helps to visually demonstrate the logic of your models.

### Code-Along: Explore the Associations in the Advertising Data

#### Visualize the relationship between the features and the response using scatterplots.

- Below, we filled in how to make a scatterplot for the columns `sales` vs `TV`. 
- Using this as an example, can you also make scatterplots for `sales` vs `radio` and `sales` vs `newspaper`?
- `axs[0]` is the first coordinate grid, `axs[1]` is the second coordinate grid, etc.

In [ ]:
# Visualize the relationship between the features and the response using scatterplots:
fig, axs = plt.subplots(1, 3, sharey=True)

advertising.plot(kind='scatter', x='TV', y='Sales', ax=axs[0], figsize=(10, 6));

#### Is there a relationship between ads and sales? Which type of ads?

In [ ]:
# Answer:

#### Can we say this a causal relationship?

In [ ]:
# Answer:

#### What other questions might we want to know about this data?

In [ ]:
# Answer:

### Group Exercise: Evaluate Which Type of Ad is Associated With Higher Sales

Let's say we want to evaluate which type of ad is associated with higher sales.

1. Draw a basic DAG on your table or whiteboard.
    - Think about other variables that may predict sales.
    - Think about confounders.
    - Think about the downstream effects changing investment in advertising.
    - Be ready to share an example.

### Section Summary

1) **It's importlant to have deep subject area knowledge.** You'll develop this over time and it will help you move through your analysis in a logical manner. However, keep in mind that you can show a strong association and still be wrong.

2) **A DAG (directed acyclic graph) can be a handy tool for thinking through the logic of your models.**

3) **There is a distinction between causation and correlation.** In our smoking example, it's relatively obvious that there's a flaw in our logic; however, this won't always be so readily apparent — especially in cutting-edge fields where there are many other unknown variables.

4) **Good data are essential.** Throughout this course we will be developing your data intuition so you can spot gaps and bias more readily. You'll also be introduced to tools that can help. However, your analysis is only as good as your understanding of the problem and the data.

<a id="sampling-bias"></a>
## Sampling Bias
---

**Sampling bias** occurs when a sample is collected in such a way that some members of the intended population are more or less likely to be included than others.

This can happen when a sample is taken non-randomly — either implicitly or explicitly.

When we have non-random sampling that results in sampling bias, it can affect the inferences or results of our analyses. We must be sure not to attribute our results to the process we observe when they could actually be because of non-random sampling.

Conceptually, this is straightforward: **When we have sampling bias, we aren't measuring what we think we are measuring.**

<a id="forms-of-sampling-bias"></a>
### Examples of Sampling Bias

- **Pre-screening:** Purposely restricting the sample to a specific group or region.
    - This typically happens when people try to study priority areas to save costs and assume priority areas are the same as random areas.
- **Self-selection:** When someone has the ability to non-randomly decide what is included in a sample.
    - This typically happens in surveys and polls but can also be an issue with other kinds of reporting.
- **Survivorship bias:** When we select only surviving subjects in a sample over time.
    - This might happen when we only look at existing customers and assume they have the same characteristics as new customers.
    
    
### Exercise: Team up and determine a potential real example of each.

<a id="problems-from-sampling-bias"></a>
### Problems That Arise From Sampling Bias
- We could overestimate or underestimate means and sample statistics for simple characteristics.
- It's possible to have artificial correlation where there should be none.

If you wanted to get the average height of the residents of Washington, DC, would you only sample the Washington Wizards?

<a id="recovering-from-sampling-bias"></a>
### Recovering From Sampling Bias
- Working out causal DAGs can help you identify when to watch out for sampling bias.
- Generally, it's best to prevent sampling bias whenever possible.
- We can't really do anything if we ENTIRELY exclude an important group of data.
- However, if portions of our data are overrepresented or underrepresented, there are ways to correct that effect.
    - Typically, we explicitly model the selection process, which means we need data on factors that determine whether  or not someone participates.
    - **Account for it**: If certain portions of the data are over/underrepresented, we can account for this.
        - The field of **survey methodology** is built around gaining insights from data collected in interesting ways.

<a id="stratified-random-sampling"></a>
### Stratified Random Sampling

Sometimes it's better to sample certain subpopulations specifically. You can perform an SRS within **strata** of the population.  This is called **stratified random sampling** and if it's done well, it can yield better insights than an SRS on the whole populations.

- **Stratified random sampling** ensures we capture important population characteristics in the random sample. If we know that the population is half males and half females, for example, we can make sure that our sample is half male and half female. We effectively break the population into two "strata" (groups), then randomly sample from each group to obtain our overall sample. This method is similar to taking a weighted average and depends on knowing key population statistics.
    - For example, if we are collecting survey data, we might ensure our participants are evenly split between men and women.

<a id="missing-data"></a>
## Missing Data
---

Sometimes we are unable to collect every attribute for a particular observation.

Unfortunately, this makes the observation unusable until we decide how to deal with it.

**We have to decide whether to:**
    - Drop the observation.
    - Drop the attribute.
    - Impute a value for that specific attribute and observation.

**So, how do we decide?**

<a id="de-minimis"></a>
### De Minimis
- If few enough observations are missing, it's not likely to change our results to a meaningful degree.
- In these cases, we don't have to bother with trivialities and simply pick a method that works well enough.

<a id="types-of-missing-data"></a>
### Types of Missing Data

- **Missing completely at random (MCAR)**
    - The reason that the data are missing is completely random and introduces no sampling bias.
    - In this case, it's safe to drop or impute.
    - We can test for this by looking at other attributes for missing and non-missing groups to see if they match.

- **Missing at random (MAR)**
    - The data are missing in a way that is related to another factor.
    - This is a form of sampling bias.
    - Like other instances of sampling bias, we can fix this by modeling the selection process.
        - This is done by building a model to impute the missing value based on other variables.

- **Missing not at random (MNAR)**
    - The response is missing in a way that relates to its own value.
    - We can't test for this.
    - We also can't fix this in a reasonable way.

<a id="class-imbalance"></a>
### Class Imbalance

Sometimes a sample may include an overrepresentation of one type of class. For example, airport security may have 990 X-ray scans showing the absence of a weapon. Due to natural scarcity, it may only provide 10 scans showing a weapon.

- If our goal is to create a model that indicates whether or not a weapon is present, then we are at a disadvantage. **Ignoring the class imbalance** would lead to a model that always guesses that a weapon is not present!
    - Note that most optimization procedures optimize for training data accuracy. Always guessing that a weapon is absent leads to 990/1000 correct results; an accuracy of 99 percent.
<br>
<br>
- A simple way to get around this is to **undersample** the majority class, deliberately leaving us with a balanced data set of 10 each. However, this is less than ideal, as it effectively ignores much of the available data.
<br>
<br>
- Alternatively, we could **oversample** the minority class by duplicating examples. Again, this is not ideal. Because we have very little data, this will magnify small differences that may just be errors, leading to a model that overfits.
<br>
<br>
Later in the course, we will look at additional methods for training models to work around class imbalance. For example, we may use an optimization algorithm that cares less about accuracy and more about minimizing particular types of errors.
<br>
<br>
<br>

<a id="introduction-to-hypothesis-testing"></a>
## Introduction to Hypothesis Testing
---

**Objective**: Test a hypothesis within a sample case study.

You'll remember that we've worked previously on descriptive statistics such as mean and variance. How would we tell if there is a difference between our groups? How would we know if this difference was real or if our finding is simply the result of chance?

After taking a random sample at Acme, men on average spend \\$60. Women on average spend \\$70 . Are these two numbers **significantly different**?
- Do we have enough information to know (No, we don't).
- Are these findingds indicative of a real trend, or are they by chance?

For example, if we are working on sales data, how would we know if there was a difference between the buying patterns of men and women at Acme, Inc.? **Hypothesis testing!**



### Examples of Hyoptheses
- The sales in California are greater than the sales in Texas.
- Customer Segment A will have a larger response than Segment B to a specific marketing strategy.
- Changing the subject line of an email will boost sales by 5%.

## Hypothesis Testing Steps

Generally speaking, we start with a **null hypothesis** and an **alternative hypothesis**, which is the opposite of the null. Then, you check whether the data support rejecting your null hypothesis or fail to reject the null hypothesis.

For example:

- **Null hypothesis**: There is no relationship between gender and sales.
- **Alternative hypothesis**: There is a relationship between gender and sales.

Note that "failing to reject" the null hypothesis is not the same as "accepting" it. Your alternative hypothesis may indeed be true, but you don't necessarily have enough data to show that yet.

This distinction is important for helping you avoid overstating your findings. You should only state what your data and analysis can truly represent.


For example:

    Null hypothesis: There is no difference between sales in California and sales in Texas.
    Alternative hypothesis:  Sales in California are greater than sales in Texas.

<a id="validate-your-findings"></a>
## Validate Your Findings

#### How Do We Tell if the Association We Observed is Statistically Significant?

**Statistical significance** is whether or not a result or relationship is caused by something other than mere random chance. Statistical hypothesis testing is traditionally employed to determine whether or not a result is statistically significant.

We might ask: **How likely is the effect observed to be true, assuming the null hypothesis is true?** If the probability of our observation occurring by chance is less than 5 percent (supposing the null hypothesis), then we reject the null hypothesis. _(Note that the 5 percent value is in many ways arbitrary)._

The probability of our observations occurring by chance, given the null hypothesis, is the **p-value** ($p$).

---

**Example:** Suppose you flip a coin three times and get three heads in a row. These three flips are our observations.

+ We want to know whether or not the coin is fair. So, we select the **null hypothesis:**
<br><br>
$$H_0: \text{The coin is fair.}$$
<br>
+ Now, let's suppose the null hypothesis is true. Three heads in a row occurs with a chance of $1/2^3 \approx 12.5\%$.
+ Because there is a reasonable ($> 5\%$) chance of three heads occuring naturally, we do not reject the null hypothesis.
+ So, **we conclude** that we do not have enough data to tell whether or not the coin is fair ($p = 0.125$).

---

In other words, we say that something is NOT statistically significant if there is a less than 5 percent chance that our finding was caused by chance alone (assuming the null hypothesis is true).

However, suppose that we flipped the coin ten times and received all 10 heads. The probability of this occuring is $1/2^10 = 1/1024$, which is below $5\%$. In this case, we **would reject the null hypothesis and conclude the coin is unfair**.

Hopefully the logic shines through here:
- 3 heads in a row isn't rare. It can happen.
- 10 heads in a row is extremely unlikely. I'd believe the coin was unfair.

<a id="confidence-intervals"></a>
### Confidence Intervals

A closely related concept is **confidence intervals**. A 95 percent confidence interval can be interpreted like so: under infinite sampling of the population, we would expect that the true value of the parameter we are estimating to fall within that range 95% of the time.

Keep in mind that we only have a **single sample of data** and not the **entire population of the data**. The "true" effect/difference is either within this interval or it is not. We have no firm knowledge, however, that our single estimate of the "true" effect/difference is close or not to the "truth". The confidence interval around our estimate tells us, with a given sample size and level of confidence, the range in which future estimates are likely to fall.

Note that using 95 percent confidence intervals is just a convention. You can create 90 percent confidence intervals (which will be more liberal), 99 percent confidence intervals (which will be more conservative), or whatever intervals you prefer.


<img src="assets/images/Conf_interval.jpeg">

<a id="error-types"></a>
### Error Types

Statisticians often classify errors not just as errors but as one of two specific types of errors — type I and type II.

+ **Type I errors** are false positives.
    - Machine learning: Our model falsely predicts "positive." (The prediction is incorrect.)
    - Statistics: Incorrect rejection of a true null hypothesis.


+ **Type II errors** are false negatives.
    - Machine learning: Our model falsely predicts "negative." (The prediction is incorrect.)
    - Statistics: Incorrectly retaining a false null hypothesis.


Understanding these errors can be especially beneficial when designing models. For example, we might decide that type I errors are OK but type II errors are not. We can then optimize our model appropriately.

> **Example:** Suppose we make a model for airline security in which we predict whether or not a weapon is present ("positive"). In this case, we would much rather have type I errors (falsely predict a weapon) than type II errors (falsely predict no weapon).

> **Example:** Suppose we make a model for the criminal justice system in which we whether or not a defendant is guilty ("positive"). In this case, we would much rather have type II errors (falsely predict innocent) than type I errors (falsely predict guilty).

<img src="assets/images/Type1_2_errors.png">


Can you phrase these examples in terms of null hypotheses?

## Kevin's Key Differences Between Business and Statistics
* **We typically ignore measurement error**- What is a customer? Timing (Database entry vs. actual time)
<br>
<br>
* **Tons of data!** Many variables and models will be significant using traditional statistical methods.
<br>
<br>
* **Time Dependency**- Businesses change over time and seasons (ice cream in winter vs. summer), whereas statistics should be absolutely repeatable. (other factors may not be apparent (confounders)
<br>
<br>
* **Business experiments generally cost a lot more** than scientific experiments, and could have greater impact.
<br>
<br>
* **Data is often incomplete** (censored or truncated)

## Case Study: Comparing Differences in Sales

<a id="scenario"></a>

---

### Scenario

You are a retail company analyzing the sales between an experiment you ran between 2 markets.

Your task is to determine if the sales between both markets are significantly different.

**Null Hypothesis:** There are no difference in the sales between markets.<br>
**Alternative Hypothesis:** The sales between the markets are different.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns

%matplotlib inline

In [ ]:
# Generate some data and randomize.
np.random.seed(42)

# Group 1
data1 = np.random.normal(85, 10, 50)

# Group 2
data2 = np.random.normal(84, 20, 50)

# Control
#data3 = np.random.normal(70, 10, 50)



# Make a DataFrame.
df = pd.DataFrame()
df["A"] = data1
df["B"] = data2
#df["C"] = data3

df.head()

In [ ]:

print(df.A.std())
print(df.B.std())
print(df.A.mean())
print(df.B.mean())

#### Plot out the distributions of group A and group B.

- Plot a histogram of ONLY the group A column, and ONLY the group B column.

In [ ]:
# Answer:
plt.hist(df.A);

In [ ]:
# Answer:
plt.hist(df.B);

#### Make a box plot of the two groups:

In [ ]:
df.boxplot();

#### Are our data sets (approximately) normal? Use the seaborn package and do a distplot.

In [ ]:
# Plot the distributions for group A and B. Are they approximately normal?
sns.distplot(df.A);
sns.distplot(df.B);

#### Let's compare the means and determine if they are significantly different.


<a id="statistical-tests"></a>
### Statistical Tests

There are a few good statistical tests for A/B testing:
* [ANOVA](https://en.wikipedia.org/wiki/Analysis_of_variance)
* [Welch's t-test](https://en.wikipedia.org/wiki/Welch's_t-test)
* [Mann-Whitney test](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test)

**Each test makes various assumptions:**
* **ANOVA** assumes the residuals are normally distributed and data have equal variances.
* **Welch's t-test** assumes normal distributions but not necessarily equal variances and more effectively accounts for small sample sizes.
* The **Mann-Whitney test** assumes nothing about the distributions but requires at least 20 data points in each set, producing a weaker p value.

Typically you need to choose the most appropriate test. Tests that make more assumptions are more discriminating (producing stronger p values) but can be misleading with data sets that don't satisfy the assumptions.


In statistics, **one-way analysis of variance** (abbreviated one-way **ANOVA**) is a technique used to compare the means of three or more samples (using the **F distribution**). The **ANOVA** tests the **null hypothesis** (the default position that there is no relationship) that samples in two or more groups are drawn from populations with the same mean values. Typically, however, the **one-way ANOVA** is used to test for differences among at least three groups, as the two-group case can be covered by a **t-test**. When there are only two means to compare, the **t-test** and the **F-test** are equivalent.

> **Note:** 
   - One-way ANOVA: An ANOVA hypothesis tests the difference in population means based on one characteristic or factor.
   - Two-way ANOVA: An ANOVA hypothesis tests comparisons between populations based on multiple characteristics.

In [ ]:
# ANOVA
F, p = stats.f_oneway(df.A, df.B)
print('F statistic:',F)
print('p-value:',p)

In [ ]:
# Welch's t-test
stats.ttest_ind(df.A, df.B, equal_var=False)

## Case Study: Comparing Differences in Sales (Alternate Data Structure)
---

### Scenario

You are a retail company analyzing the sales between an experiment you ran between 2 markets.

Your task is to determine if the sales between both markets are significantly different.

**Null Hypothesis:** There are no difference in the sales between markets.<br>
**Alternative Hypothesis:** The sales between the markets are different.

In [ ]:
np.random.seed(42)

# Generate some data and randomize.

# Group 1
data1 = np.random.normal(85, 10, 1000)
groupA = pd.DataFrame()
groupA['sales']=data1
groupA['group']='A'
# Group 2
data2 = np.random.normal(83, 10, 1000)
groupB = pd.DataFrame()
groupB['sales']=data2
groupB['group']='B'
# Control
data3 = np.random.normal(70, 10, 100)



# Make a DataFrame.
df = groupA
df = df.append(groupB)



df

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

sales_lm = ols('sales ~ group',
                data=df).fit()

table = sm.stats.anova_lm(sales_lm, typ=2)
print(table)

## Class Challenge: A/B Testing Hypothesis Tests

<a id="scenario"></a>

---

### Scenario

You are a data science team working for a web-based company and you are planning to roll out a new website design. One of two competing designs were presented to random samples of users, and their ultimate purchase total was recorded (if any).

Your task is to determine which of the two designs yields higher total purchases and if the result is statistically significant.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns

%matplotlib inline
np.random.seed(42)

In [ ]:
# Generate some data and randomize.

# Some people bought nothing, the others bought 
# with some distribution.
data1 = [0] * 50
data1.extend(np.random.normal(14, 4, 150))
np.random.shuffle(data1)

# The second design hooked less people, 
# but those that were hooked bought more stuff.
data2 = [0] * 100
data2.extend(np.random.normal(20, 5, 100))
np.random.shuffle(data2)

# Make a DataFrame.
df = pd.DataFrame()
df["A"] = data1
df["B"] = data2

df.head() 

#### Plot out the distributions of group A and group B.

- Plot a histogram of ONLY the group A column, and ONLY the group B column.

In [ ]:
# Answer:

In [ ]:
# Answer:

#### Make a box plot of the two groups:

In [ ]:
# Plot the boxplot:

#### Are our data sets (approximately) normal? Use what we learned in the previous lesson to decide:

In [ ]:
# Plot the distributions for group A and B. Are they approximately normal?

#### Which test is most appropriate for our data?

In [ ]:
# Answer:

#### Use the Mann-Whitney test on our data.

- Look up the function in SciPy [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html).
- Statistic: Float the Mann-Whitney U statistic — equal to min(U for x, U for y) if alternative is equal to none (deprecated; exists for backward compatibility) — and U for Y otherwise.
- P value: Float p value assuming an asymptotic normal distribution — one sided or two sided, depending on the choice of alternative.

In [ ]:
# Answer:

The Mann–Whitney U test (also called the Mann–Whitney–Wilcoxon (MWW), Wilcoxon rank-sum test, or Wilcoxon–Mann–Whitney test) is a nonparametric test of the null hypothesis of whether it is equally likely that a randomly selected value from one sample will be less than or greater than a randomly selected value from a second sample.

Unlike the t-test, it does not require the assumption of normal distributions. It is also nearly as efficient as the t-test on normal distributions.

<a id="interpret-your-results"></a>
### Interpret Your Results
* Is there a significant difference in the mean total purchases in the two designs?
* Which design do you recommend? Why? 
* Write two sentences explaining your results and your recommendation.

In [ ]:
# Answer:

## Conclusions/Summary

* **Hypothesis Testing**: What is it used for?
    - It's often used for _differentiating things_. Today we used it to tell apart two categories. We'll soon use HTs for another purpose: model selection.
* **Confidence Intervals**: What are they used for?
    - Giving ranges of feasible values. When HTs answer yes/no questions, CIs answer open-ended, quantitative ones.

## Extra Practice: Acme Shopping

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns

%matplotlib inline

In [ ]:
# Generating Data
np.random.seed(123)
df_m = pd.DataFrame({
    'sex': 'M',
    'amount': np.random.normal(loc=60, scale=3, size=100)
})

df_f = pd.DataFrame({
    'sex': 'F',
    'amount': np.random.normal(loc=70, scale=4, size=100)
})

df = pd.concat([df_m, df_f], axis=0)
df.head()

#### Research Question:
Are the spending amounts of men and women different at Acme?

In [ ]:
# Plot the data for each sex. What do you see?
df.hist(column='amount', by='sex');

In [ ]:
# Formulate a hypothesis test:


In [ ]:
# Run a t-test

In [ ]:
#Make a conclusion